Import Libraries

In [11]:
import sqlite3
import requests
from tqdm import tqdm

from flask import Flask, request
import json 
import numpy as np
import pandas as pd

# Introduction
In most cases when we are running a business, there are a lot of data stakeholder outside our company. The problem is that we need to provide the access in a way that they will not break our security rules or concerns. One way to solve that is by creating an API for the database. In this project, we will introduce you on how python is used for data transaction management using Flask API. 

**Usecase**: Bikeshare App\
Have you ever rent a bike for faster mobility in town? In the past few years, this business once become a phenomenom. In Indonesia, there are lots of similar services, for example, the Jakarta government's "GOWES" bike sharing service that launcehd in July 2020. 

For the user perspective, the general journey is denoted as follows:
- User scan the bike located at some bike station, sending the data to database as the intent of "start riding"
- Once user has reached its destination station, he/she put back the bike, sending the data again to the database as the intent of "finished riding"

For each activity, there are data transactions between user and database. And how do you think each user's phone communicate with the server for storing and receiving the data? Using API ofcourse! 

We will later create a simplified version of the API service which handles data transactions and analysis. 


**Goals**: Make an API service to connect 3rd party user with data using HTTP request

**End Product**: A Flask API which capable of doing: 
- Input new data to database
- Read specific data from database
- Get specific insight from data analysis process (ie: best performing stations)

**Scoring Metrics**: 

1. 1 point - Created Flask App

-  create app.py file to make flask app
-  create Flask app to execute all of the endpoint you have made

2. 2 points - Created functionality to read or get specific data from the database

- create query to read data from database
- create function to execute read specific information into table from database

3. 4 points - Created functionality to input new data into each table for the databases

- create query to insert new data into stations and trips table
- create function to execute input data into stations and trips table

4. 3 points - Created static endpoints which return analytical result (must be different from point 2,3)

- create query to make analytical resultfrom the data
- create static endpoint to analyze the data from database, for example average trip durations
    
5. 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)

- create query to make analytical resultfrom the data
- create dinamic endpoint to analyze the data from database, for example average trip durations for each bike_id

6. 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)

- create input data for refering into query for post endpoint
- make query and aggregation function to implement the input

**Tools**: 
- **Python** with **Jupyter Notebook**, installed with **required libraries**
- **Visual Studio Code (VSCode)**: Recommended for writing application scripts
- **TablePlus**: Recommended for easy database access and exploration
- Postman: Optional for easy API testing


# About the Data

The data that we will be using in this project is **[Austin Bike Share]()** dataset which contains information on bike trip start location, stop location, duration, type of bike share user in the city of Austin, Texas. Bike station location data is also provided. 

All the information is stored in a database called **austin_bikeshare.db**. However, we also provides the non existing data in csv files. These data then will be imported into the database using the API

Lists of files:
- **austin_bikeshare.db**: The database, contains `trips` and `stations` table
- **data/austin_bikeshare_stations.csv**: contains all the stations information which is not yet available in the database table
- **data/austin_bikeshare_trips_2021.csv**: contains all 2021 data which is not yet available in the database table

In [12]:
# Reading the csv data
trips = pd.read_csv('data/austin_bikeshare_trips_2021.csv')
stations = pd.read_csv('data/austin_bikeshare_stations.csv')

## Taking a look for trips data
`trips` table in database, or `austin_bikeshare_trips.csv` in original files. It roughly consisted of 1.3 million rows

In [13]:
trips.head(2)

,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,23455589,Local365,174,2021-01-26 17:47:42 UTC,4059.0,Nash Hernandez/East @ RBJ South,3660.0,East 6th/Medina,7
1,23459960,Local365,19265,2021-01-28 08:03:52 UTC,4054.0,Rosewood/Chicon,4055.0,11th/Salina,16


**Data Descriptions**:
- `bikeid`: integer id of bike
- `checkout_time`: HH:MM:SS, see start time for date stamp
- `duration_minutes`: int minutes of trip duration
- `endstationid`: integer id of end station
- `endstationname`: string of end station name
- `month`: month, integer
- `startstationid`: integer id of start station
- `startstationname`: string of start station name
- `start_time`: YYYY-MM-DD HH:MM:SS
- `subscriber_type`: membership typ e.g. walk up, annual, other bike share, etc
- `trip_id`: unique trip id int
- `year`: year of trip, int

## Taking a look for stations data
`stations` table in database, or `austin_bikeshare_stations.csv` in original files 

In [14]:
stations.tail(2)

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
96,3795,Dean Keeton & Whitis,active,300 W. Dean Keeton St.,NaN,32671.0,undetermined_parking,19.0,solar,55.0,5.0,NaN,9,2021-01-04T12:00:00Z
97,3797,21st & University,active,2007 University Ave.,NaN,32674.0,undetermined_parking,19.0,solar,55.0,5.0,ROW might belong to UT,9,2021-01-04T12:00:00Z


**stations table (or austin_bikeshare_trips.csv)**:
- `station_id`: integer id of station
- `name`: string of station name
- `status`: string of station status (active, closed, moved, ACL-only)
- `address`: string of station address
- `alternate_name`: string of station alternative name
- `city_asset_number`: integer of station's asset number
- `property_type`: string of station's property type
- `number_of_docks`: integer of number of available bike docks
- `power_type`: string of station's power source type
- `footprint_length`: float of station' blueprint length (size of the station). Probably in meters
- `footprint_width`: float of station' blueprint width (size of the station). Probably in meters
- `notes`: string of additional notes
- `council_district`: integer of stations's council district
- `modified_date`: date of last modified information regarding the station

# Database

The first important task is to make sure we can securely make connections to the database. In this scenario, we will connect to sqlite database, `austin_bikeshare.db` via python. In this part, we will re-visit on how to work with databases, started with making a connection, get some data, and insert data into it. 

You can directly connect and view the database using TablePlus, or run the following code to create the connection

In [15]:
# Define a function to create connection for reusability purpose
def make_connection():
    connection = sqlite3.connect('austin_bikeshare.db')
    return connection

# Make a connection
conn = make_connection()

## POST: Insert data into database

Generally, POST method will utilize the data sent by user for specific purporse, for example: 

- Insert new data into the database. 
- Operate the data into some function

We will learn how to insert data into a specific table in our database. Please refer to the following code to create and run the query for the given task

In [16]:
data = tuple(stations.iloc[1].fillna('').values)
data

(2500,
 'Republic Square',
 'closed',
 '425 W 4th Street',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 9,
 '2021-01-04T12:00:00Z')

In [17]:
# Get the data values
data = tuple(stations.iloc[1].fillna('').values)

# Make the query
query = f"""
INSERT INTO stations
VALUES {data}
"""

In [18]:
# See the actual query looks like
print(query)


INSERT INTO stations
VALUES (2500, 'Republic Square', 'closed', '425 W 4th Street', '', '', '', '', '', '', '', '', 9, '2021-01-04T12:00:00Z')



In [19]:
# Execute the query 
conn.execute(query)

IntegrityError: UNIQUE constraint failed: stations.station_id

In [ ]:
# Commit changes to database
conn.commit()

Once the code above succesful, it's recomended to wrap it into a function so that we can reuse it in the future. Complete the following codes to make the function:

In [20]:
def insert_into_stations(data, conn):
    query = f"""INSERT INTO stations values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'

In [23]:
# Example use of the function 
conn = make_connection()
data = tuple(stations.iloc[10].fillna('').values) # Randomly select a data
result = insert_into_stations(data, conn)

In [24]:
result

'Error'

**TASK: Create a Function to insert data for `trips` table**

In [ ]:
# Your code here
def insert_into_trips(data, conn):
    query = f"""INSERT INTO trips values {data}"""
    try:
        conn.execute(query)
    except:
        return 'Error'
    conn.commit()
    return 'OK'


## GET: Read specific data

Generally, GET method will ask for specific the data in the database, alongside with additional information we might send. For example: 
- Get number of unique user
- Get full trips information for specific user id
- Get average trips duration and length for specific user id

In this example, we will implement the most basic GET method, that is to get Station information in the table based on specific Station ID

In [ ]:
# Get Specific Station ID  Query
station_id = 2500
query_station_id = f"""
SELECT * FROM stations
WHERE station_id = {station_id}
"""

# Get all station ID Query
query_station_all = "SELECT * FROM stations"

In [ ]:
print(query_station_id)


SELECT * FROM stations
WHERE station_id = 2500



In [ ]:
# Run the query and get the result
result = pd.read_sql_query(query_station_id, conn)

In [ ]:
result

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


Just like the previous section, create a function to read specific station

In [ ]:
def get_station_id(station_id, conn):
    query = f"""SELECT * FROM stations WHERE station_id = {station_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result

**TASK: Create a Function to read data from `trips` table**

In [ ]:
# Your code here
def get_trip_id(trip_id, conn):
    query = f"""SELECT * FROM trips WHERE id = {trip_id}"""
    result = pd.read_sql_query(query, conn)
    return result 

    
def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result


In [ ]:
pd.read_sql_query("""SELECT * FROM trips limit 5""", conn)

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
0,9900289692,Walk Up,248,2015-10-02 21:12:01 UTC,1006,Zilker Park West,1008,Nueces @ 3rd,39
1,9900285987,24-Hour Kiosk (Austin B-cycle),446,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
2,9900285989,24-Hour Kiosk (Austin B-cycle),203,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,31
3,9900285991,24-Hour Kiosk (Austin B-cycle),101,2014-10-26 15:12:00 UTC,2712,Toomey Rd @ South Lamar,2712,Toomey Rd @ South Lamar,30
4,9900286140,24-Hour Kiosk (Austin B-cycle),242,2014-10-26 18:12:00 UTC,2541,State Capitol @ 14th & Colorado,2541,State Capitol @ 14th & Colorado,19


# Flask App

Flask is a micro-framework for python. Generally we can build any application out of it. To start with, let's make our first empy flask app. 
Creata a new file `app.py`, then copy-paste the following codes into it and save it. 

*Notes: it's recomended to use vscode as editor since it support vast linting supports, including python which we find really helpful in coding*

```python
from flask import Flask, request
app = Flask(__name__) 

if __name__ == '__main__':
    app.run(debug=True, port=5000)
```

To run the app, you can open your terminal, go to the specific folder and run `python app.py` using your designated virtual environment

## Routes and Endpoints

### Implement endpoints

If you succesfully run the previous app and access the localhost:5000 , you might get an 404 not found error. This is because we **have not** define yet what will happen if people accessing our root path of the app (localhost:5000/)

Add the following example of route or endpoint into your `app.py` just before the `if __name__ === '__main__':` line, and reload the page to see if it works

```python 
@app.route('/')
def home():
    return 'Hello World'
```

Above endpoints will runs the `home` function anytime user access the `/` page.

Since we are going to handle all the requests through this app, the very next step is to implement our functionalites. Let's start with implementing the functionality to read all station data

we will add the `get_all_stations()` functions into our app, create a `@app.route('/stations/')` endpoint and call the function in it. The code should looks like:

```python 
@app.route('/stations/')
def route_all_stations():
    conn = make_connection()
    stations = get_all_stations(conn)
    return stations.to_json()
    
def get_all_stations(conn):
    query = f"""SELECT * FROM stations"""
    result = pd.read_sql_query(query, conn)
    return result
```

However, adding just above codes is not enough. If you see, there is a dependency inside the function, which is `make_connection()` function. Other than that, we will be using all the required libraries. 

Hence, we can complete the code by : 
- import the required libraries at the top of the `app.py` file
- write the `make_connection()` function before any routes declarations

Once it's completed, you should see no errors in the vscode screen, and your endpoint should work just fine and returns all the stations. It's now your task to impelement the `get_all_trips` into the app 

**TASK: Implement `get_all_trips()` method into the app** \
Place it under `@app.route('/trips/')` endpoint

In [ ]:

def get_all_trips(conn):
    query = f"""SELECT * FROM trips"""
    result = pd.read_sql_query(query, conn)
    return result

### Access Endpoints

To access our endpoints using python, simply use `requests` library followed by the request method (post, get, put, delete, etc) the passed in the url or data 

In [ ]:
url = 'http://127.0.0.1:5000/stations/'
res = requests.get(url)

In [ ]:
res

<Response [200]>

The response sent by the server was not only contains bare data. It was packed as HTTP response, meaning that we need to unpack the response first  in order to get the data. we can use `res.json()` to obtain it, then use pandas to transform it into dataframe for readibility or future analysis purposes. 

In [ ]:
pd.DataFrame(res.json())#.head()

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z
1,2546,ACC - West & 12th Street,closed,1231 West Ave.,,,,,,,,,9,2021-01-04T12:00:00Z


## Static and Dynamic Endpoints

On previous part we already made several endpoints which control how our server will react whenever users access it. 
If we take a look at the endpoints, it's all static : 
- `@app.route('/')`
- `@app.route('/home/')`
- `@app.route('/stations/')`
- `@app.route('/trips/')`


What if, instead of getting all the stations information, we only wanted to read a specific station information of station_id 3464 ? \
should we make an excact endpoint of that like `@app.route('stations/3464')`?. But what about the others? are we going to make a bunch of endpoints for each specific id like: 
- `@app.route('/stations/3464')`
- `@app.route('/stations/2500')`
- `@app.route('/stations/2541')`
- and so on...

Of course we shouldn't. One way to overcome the problem is by delcaring a **dynamic endpoints**. Basically, it's an endpoint which allows user to insert a variable values in it. The final looks of the dynamic endpoint version of above problem is: 
- `@app.route('stations/<station_id>')`

From above endpoint, the \<station_id\> is the variable, and we need to make sure that it influence how our following function acts. The whole dynamic endpoints for getting specific station by its id will looks like: 

```python 
@app.route('/stations/<station_id>')
def route_stations_id(station_id):
    conn = make_connection()
    station = get_station_id(station_id, conn)
    return station.to_json()
```


Before accesing the endpoint, make sure that `make_connection()` and `get_station_id()` functions are included inside `app.py` script so that it won't raise an error. Now for the final step, we can try to access the dynamic endpoints by changing the `station_id` in the following codes

In [ ]:
station_id = 2500
url = f"http://127.0.0.1:5000/stations/{station_id}"

In [ ]:
response = requests.get(url)
pd.DataFrame(response.json())

,station_id,name,status,address,alternate_name,city_asset_number,property_type,number_of_docks,power_type,footprint_length,footprint_width,notes,council_district,modified_date
0,2500,Republic Square,closed,425 W 4th Street,,,,,,,,,9,2021-01-04T12:00:00Z


**TASK: Implement dynamic endpoints to read specific trip by its trip_id!** \
Place it under `@app.route('/trips/<trip_id>')` endpoint

## Handling JSON data as input

Sometimes, in order to make something happens in our API, we need the user to send us the data. In this case, we need to handle how we can get the data (which mostly sent as json format) and utilize it inside our endpoint functions. 

In order to achieve that, we will be using flask's `request` classes.


```python
@app.route('/json') 
def json_example():
    
    req = request.get_json(force=True) # Parse the incoming json data as Dictionary
    
    name = req['name']
    age = req['age']
    address = req['address']
    
    return (f'''Hello {name}, your age is {age}, and your address in {address}
            ''')
```

In [ ]:
data = {
    "name" : "monyet", 
    "age" : 24,
    "address" : "pohon tomat"
}

url = "http://127.0.0.1:5000/json"

In [ ]:
res = requests.post(url, json=data)

In [ ]:
res

<Response [200]>

In [ ]:
res.text

'Hello monyet, your age is 24, and your address in pohon tomat\n            '

Now that we already know how to handle json input, we can try to implement and endpoint in which we can insert new data into `stations` table. 

```python

@app.route('/stations/add', methods=['POST']) 
def route_add_station():
    # parse and transform incoming data into a tuple as we need 
    data = pd.Series(eval(request.get_json(force=True))) #eval agar pandast tahu ini representasi json string yg dikirim lwt internet
    data = tuple(data.fillna('').values)
    conn = make_connection()
    result = insert_into_stations(data, conn)
    return result


```

In [ ]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

In [ ]:
data = stations.iloc[20].fillna('').to_dict()
data_json = json.dumps(data, cls=NpEncoder)
data_json

'{"station_id": 2502, "name": "Barton Springs & Riverside", "status": "closed", "address": "400 Barton Springs Rd.", "alternate_name": "", "city_asset_number": "", "property_type": " ", "number_of_docks": "", "power_type": " ", "footprint_length": "", "footprint_width": "", "notes": "moved to Pedernales/2nd. Original city ID: 16729", "council_district": 9, "modified_date": "2021-01-04T12:00:00Z"}'

In [ ]:
url = "http://127.0.0.1:5000/stations/add"
res = requests.post(url, json=data_json)

In [ ]:
res

<Response [200]>

In [ ]:
res.text

'OK'

In [ ]:

def route_add_trips():
    data = pd.Series(eval(request.get_json(force=True))) 
    data = tuple(data.fillna('').values)
    conn = make_connection()
    result = insert_into_trips(data, conn)
    return result

In [ ]:
# Sintaks for iteratively insert stations (from stations_csv) into stations table

for idx, data in tqdm(stations.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    url = "http://127.0.0.1:5000/stations/add"
    res = requests.post(url, json=data_json)

98it [00:00, 243.80it/s]


**TASK: Using the API, insert all the trips data (in csv) into the database by running the following codes**\
It might run differently on each devices, so you might get some rest while waiting for it to complete

In [ ]:
for idx, data in tqdm(trips.iterrows()):
    data = data.fillna('').to_dict()
    data_json = json.dumps(data, cls=NpEncoder)
    url = "http://127.0.0.1:5000/trips/add"
    res = requests.post(url, json=data_json)

17982it [00:55, 324.99it/s]


In [ ]:
res

<Response [200]>

# Make your own analytic endpoints

Based on what we've learned before, create your own endpoints which returns analytical result such as contigency tables, aggregation tables, or even just a values.

*ps: if the return is dataframe/series, don't forget to change it into json with `.to_json()` method*


- [x] 1 point - Created Flask App
- [x] 4 points - Created functionality to input new data into each table for the databases
- [x] 2 poitns - Created functionality to read or get specific data from the database
- [ ] 3 points - Created static endpoints which return analytical result (must be different from point 2,3)
- [ ] 3 points - Created dynamic endpoints which return analytical result (must be different from point 2,3,4)
- [ ] 3 points - Created POST endpoint which receive input data, then utilize it to get analytical result (must be different from point 2,3,4,5)


## Create Static Endpoint(s)

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [ ]:
# Your Code Here 
@app.route('/trips/average_duration') 

In [ ]:
def avg_duration(trips):
    avg_dur = trips.groupby("subscriber_type")["duration_minutes"].mean().sort_values()
    return avg_dur.to_json()

In [ ]:
def trips_avg_duration():
    conn = make_connection()
    trips = get_all_trips(conn)
    table = avg_duration(trips)
    return table

In [ ]:
trips_avg_duration()

'{"PROHIBITED":0.25,"RESTRICTED":2.0,"Annual (Broward B-cycle)":6.485915493,"RideScout Single Ride":6.5050251256,"Local365 Youth (age 13-17 riders)- 1\\/2 off Special":7.0,"Annual (Omaha B-cycle)":7.6666666667,"Annual Pass (30 minute)":8.5,"UT Student Membership":8.9,"Founding Member (Austin B-cycle)":9.2163531114,"Denver B-cycle Founder":9.3,"Membership: pay once  one-year commitment":9.4,"Annual Membership (Austin B-cycle)":9.6842869399,"Founding Member":9.8281690141,"Annual Membership (Charlotte B-cycle)":10.1818181818,"Semester Membership":10.2922222222,"Semester Membership (Austin B-cycle)":10.8985985161,"Aluminum Access":11.0396039604,"Annual Membership ":11.3333333333,"Local30 ($11 plus tax)":12.055952381,"Annual (Kansas City B-cycle)":12.1960784314,"Local365 ($80 plus tax)":13.1036204744,"Annual Membership (Indy - Pacers Bikeshare )":13.5,"Annual (San Antonio B-cycle)":13.6119402985,"7-Day":14.2684093083,"7-Day Membership (Austin B-cycle)":14.4782608696,"Annual (Boulder B-cycle

In [ ]:
url = f"http://127.0.0.1:5000/trips/average_duration"
res = requests.get(url)
res

<Response [200]>

## Create Dynamic Endpoints 

You can use the following cell to try-out your function before implementing it as an endpoint.

After created the endpoint, make sure to implement it to `app.py` file.

In [ ]:
# Your Code Here
@app.route('/trips/average_duration/<bike_id>')

In [ ]:
def trips_bikeid(bike_id):
    conn = make_connection()
    bikeid = get_bike_id(bike_id, conn)
    return bikeid


In [ ]:
def get_bike_id(bike_id, conn):
    query = f"""SELECT * FROM trips WHERE bikeid = {bike_id}"""
    result = pd.read_sql_query(query, conn)
    avg_dur = result.groupby("subscriber_type")["duration_minutes"].mean()
    return avg_dur.to_json()

In [140]:
bike_id=406
url = f"http://127.0.0.1:5000/trips/average_duration/{bike_id}"
res = requests.get(url)
res

<Response [200]>

## Create POST Endpoints

You can use the following cell to try-out your function before implementing it as an endpoint.

If you still find it quite difficult, here's an example case you might wanted to try on:
- input : a dictionary contained a datetime period
    - { "period" : "2015-08" }
- output:
    - Aggregation table of bike rent activities for each station in that specific period
- example code: 

```python 
input_data = request.get_json() # Get the input as dictionary
specified_date = input_data['period'] # Select specific items (period) from the dictionary (the value will be "2015-08")

# Subset the data with query 
conn = make_connection()
query = f"SELECT * FROM stations WHERE start_time LIKE ({specified_date}%)"
selected_data = pd.read_sql_query(query, conn)

# Make the aggregate
result = selected_data.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'
})

# Return the result
return result.to_json()

```

After created the endpoint, make sure to implement it to app.py file.

In [25]:
#@app.route('/aggregation', methods=['POST']) 
def input_time():
    req = request.get_json(force=True) 
    start = req['start']
    end = req['end']
    return (f'''Time filtered is from {start} to {end}
            ''')

In [31]:
trips.head(2)

start = "2021-01-25"
end = "2021-01-26"
trips[(trips["start_time"] >= start) & (trips["start_time"] <= end)]


,trip_id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes
70,23451688,Local31,17635,2021-01-25 15:57:43 UTC,4055.0,11th/Salina,3291.0,11th/San Jacinto,7
126,23452411,Local31,17635,2021-01-25 18:52:32 UTC,3291.0,11th/San Jacinto,2561.0,12th/San Jacinto @ State Capitol Visitors Garage,44
150,23452652,Single Trip (Pay-as-you-ride),136G,2021-01-25 20:21:23 UTC,3791.0,Lake Austin/Enfield,3791.0,Lake Austin/Enfield,29
151,23452755,Pay-as-you-ride,666,2021-01-25 21:31:33 UTC,4054.0,Rosewood/Chicon,4057.0,6th/Chalmers,6
152,23452858,Pay-as-you-ride,18696,2021-01-25 22:48:51 UTC,4057.0,6th/Chalmers,4054.0,Rosewood/Chicon,5
...,...,...,...,...,...,...,...,...,...
17906,23451519,Local31,19213,2021-01-25 15:24:41 UTC,3838.0,26th/Nueces,3838.0,26th/Nueces,42
17907,23450763,Local31,21447,2021-01-25 12:22:25 UTC,3838.0,26th/Nueces,3838.0,26th/Nueces,34
17918,23451413,Local31,21447,2021-01-25 15:03:31 UTC,3838.0,26th/Nueces,3838.0,26th/Nueces,20
17930,23449891,Local365,16874,2021-01-25 00:48:21 UTC,3838.0,26th/Nueces,3838.0,26th/Nueces,7


In [34]:
start = "2021-01-25"
end = "2021-01-26"

def aggregate(start, end, conn):
    query = f"""SELECT t.*, s.station_id, s.name, s.status, s.address FROM trips AS t LEFT JOIN stations AS s ON t.start_station_id = s.station_id WHERE t.start_time BETWEEN '{start}' AND '{end}'"""
    result = pd.read_sql_query(query, conn)
    result = result.groupby('start_station_id').agg({
    'bikeid' : 'count', 
    'duration_minutes' : 'mean'})
    return result

,id,subscriber_type,bikeid,start_time,start_station_id,start_station_name,end_station_id,end_station_name,duration_minutes,station_id,name,status,address
0,23451688,Local31,17635,2021-01-25 15:57:43 UTC,4055,11th/Salina,3291,11th/San Jacinto,7,4055,11th & Salina,active,1705 E 11th St
1,23452411,Local31,17635,2021-01-25 18:52:32 UTC,3291,11th/San Jacinto,2561,12th/San Jacinto @ State Capitol Visitors Garage,44,3291,11th & San Jacinto,active,310 E. 11th St.
2,23452652,Single Trip (Pay-as-you-ride),136G,2021-01-25 20:21:23 UTC,3791,Lake Austin/Enfield,3791,Lake Austin/Enfield,29,3791,Lake Austin & Enfield,active,3825 Lake Austin Blvd
3,23452755,Pay-as-you-ride,666,2021-01-25 21:31:33 UTC,4054,Rosewood/Chicon,4057,6th/Chalmers,6,4054,Rosewood & Chicon,active,1909 Rosewood Avenue
4,23452858,Pay-as-you-ride,18696,2021-01-25 22:48:51 UTC,4057,6th/Chalmers,4054,Rosewood/Chicon,5,4057,6th & Chalmers,active,639 Chalmers Avenue
...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,23451519,Local31,19213,2021-01-25 15:24:41 UTC,3838,26th/Nueces,3838,26th/Nueces,42,3838,Nueces & 26th,active,600 W. 26th St.
373,23450763,Local31,21447,2021-01-25 12:22:25 UTC,3838,26th/Nueces,3838,26th/Nueces,34,3838,Nueces & 26th,active,600 W. 26th St.
374,23451413,Local31,21447,2021-01-25 15:03:31 UTC,3838,26th/Nueces,3838,26th/Nueces,20,3838,Nueces & 26th,active,600 W. 26th St.
375,23449891,Local365,16874,2021-01-25 00:48:21 UTC,3838,26th/Nueces,3838,26th/Nueces,7,3838,Nueces & 26th,active,600 W. 26th St.


In [35]:
#testing

url = "http://127.0.0.1:5000/aggregation"
date_data = {
    "start" : "2021-01-25", 
    "end" : "2021-01-26"
}


In [39]:
res = requests.post(url, json=date_data)
res

<Response [200]>

In [38]:
res.text

'{"bikeid":{"2494":4,"2495":3,"2496":3,"2497":12,"2498":9,"2499":4,"2501":12,"2503":1,"2504":3,"2537":3,"2539":13,"2542":4,"2544":1,"2547":11,"2548":19,"2549":10,"2552":17,"2561":2,"2562":1,"2563":3,"2565":1,"2566":7,"2567":9,"2568":2,"2571":1,"2572":6,"2574":3,"2575":6,"2707":10,"2711":1,"2823":2,"3291":6,"3292":2,"3293":1,"3377":4,"3390":2,"3455":5,"3513":4,"3619":4,"3621":6,"3660":2,"3684":4,"3685":1,"3686":4,"3687":4,"3790":5,"3791":6,"3792":11,"3793":14,"3794":1,"3795":5,"3797":7,"3798":17,"3799":12,"3838":17,"3841":10,"4047":1,"4048":3,"4050":4,"4051":1,"4054":4,"4055":2,"4057":5,"4058":1,"4059":3,"4060":1,"4061":8,"4062":7},"duration_minutes":{"2494":23.5,"2495":5.6666666667,"2496":29.6666666667,"2497":28.25,"2498":14.7777777778,"2499":22.0,"2501":22.6666666667,"2503":54.0,"2504":29.0,"2537":33.3333333333,"2539":33.7692307692,"2542":28.0,"2544":12.0,"2547":6.4545454545,"2548":20.6315789474,"2549":42.7,"2552":24.4705882353,"2561":15.5,"2562":2.0,"2563":8.3333333333,"2565":17.0,"2

# Submission

After finishing your work of all the rubrics, the next step will be;

1. Prepare your `Bikeshare API.ipynb` file that has been edited with your code and wrangling data.
2. Prepare your `app.py` file for your Flask App. Make sure you have implemented all the endpoints to `app.py` (including your custom static, dynamic and post endpoints).
3. Submit your `Bikeshare API.ipynb` and `app.py` file in your github repository. The dataset is optional to post since it has big size to post. 

